# Dependencies


In [1]:
!pip install fire==0.5.0
!pip install pydantic==1.10.6
!pip install pytorch-lightning==2.0.0
!pip install torch==2.0.0
!pip install tqdm==4.65.0
!pip install transformers==4.28.1
!pip install safetensors==0.3.0
!pip install datasets==2.11.0
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install peft
!pip install sacrebleu
!pip install data_loading

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=53a48f77117e65c93e1dd53b80515c1e8381a1cf9043b3ad686ab95ef2e8caf8
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 16.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached n

# Imports

In [2]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Adafactor
from pytorch_lightning import seed_everything
import argparse
import functools
import os
import pytorch_lightning as pl
import torch
from peft import LoraConfig, TaskType, get_peft_model
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.strategies import FSDPStrategy
from torch.distributed.fsdp import ( MixedPrecision,FullyShardedDataParallel,StateDictType, FullStateDictConfig,)
from torch.distributed.fsdp.wrap import transformer_auto_wrap_policy
from transformers.models.t5.modeling_t5 import T5Block
from torch.utils.data import DataLoader
import json
import random
from collections import Counter
from pathlib import Path
from typing import List, Optional, Dict

from datasets import load_dataset
from fire import Fire
from pydantic import BaseModel, Field
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import PreTrainedTokenizer, BatchEncoding, AutoTokenizer

# Preprocessing Dataset

In [3]:
class FindingTokensAnalyze(BaseModel, arbitrary_types_allowed=True):
    name: str
    tokenizer: Optional[PreTrainedTokenizer]

    def load(self):
        if self.tokenizer is None:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.name, model_max_length=99999
            )

    def run(self, texts: List[str], limit: int = 0) -> Dict[str, float]:
        if limit:
            texts = texts[:limit]

        self.load()
        tokens = self.tokenizer(texts).input_ids
        lengths = sorted(len(lst) for lst in tokens)
        info = dict(min=lengths[0], max=lengths[-1], median=lengths[len(lengths) // 2])
        info.update({"95_percentile": lengths[round(len(lengths) * 0.95)]})
        return info


class TextToTextSample(BaseModel):
    source: str
    target: str


class TextToTextData(BaseModel):
    samples: List[TextToTextSample]

    @classmethod
    def load(cls, path: str):
        with open(path) as f:
            all_lines = tqdm(f.readlines(), desc=path)
            samples = [TextToTextSample(**json.loads(line)) for line in all_lines]
        return cls(samples=samples)

    def save(self, path: str):
        Path(path).parent.mkdir(exist_ok=True, parents=True)
        with open(path, "w") as f:
            for sample in self.samples:
                print(sample.json(), file=f)

    def analyze(self, num: int = 10, tokenizer_name: str = "t5-base"):
        random.seed(num)
        for sample in random.sample(self.samples, k=num):
            print(sample.json(indent=2))

        token_checker = FindingTokensAnalyze(name=tokenizer_name)
        info = dict(
            total_samples=len(self.samples),
            source=str(token_checker.run([sample.source for sample in self.samples])),
            target=str(token_checker.run([sample.target for sample in self.samples])),
        )
        print(json.dumps(info, indent=2))


class MentalSample(BaseModel):
    Context: str
    Response: str


class Mental_Data(BaseModel):
    samples: List[MentalSample]

    @classmethod
    def load(cls, path: str):
        with open(path) as f:
            raw = json.load(f)
            return cls(samples=[MentalSample(**r) for r in raw])

    def save(self, path: str):
        raw = [sample.dict() for sample in self.samples]
        Path(path).parent.mkdir(exist_ok=True, parents=True)
        with open(path, "w") as f:
            json.dump(raw, f)

    def as_data(self) -> TextToTextData:
        self.analyze()
        samples = []
        for raw in self.samples:
            # source = raw.instruction.strip()
            if raw.Context.strip():
                source = raw.Context
            samples.append(TextToTextSample(source=source, target=raw.Response))
        return TextToTextData(samples=samples)

    def analyze(self):
        info = dict(
            Context_Sample=len(self.samples),
            Response_Sample=sum(sample.Context.strip() != "" for sample in self.samples),
        )
        print(json.dumps(info, indent=2))


class TextToTextDataset(Dataset):
    def _init_(
        self,
        path: str,
        tokenizer: PreTrainedTokenizer,
        max_source_length: int,
        max_target_length: int,
    ):
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer
        self.data = TextToTextData.load(path)

    def _len_(self) -> int:
        return len(self.data.samples)

    def tokenize(self, text: str, is_source: bool) -> BatchEncoding:
        x = self.tokenizer(
            text,
            max_length=self.max_source_length if is_source else self.max_target_length,
            padding="max_length",
            truncation=not is_source,
            return_tensors="pt",
        )

        """
        T5 truncates on right by default, but we can easily truncate on left
        for the encoder input as there is no special token on the left side
        """
        if is_source:
            assert x.input_ids.ndim == 2
            assert x.input_ids.shape == x.attention_mask.shape
            length = x.input_ids.shape[1]
            start = max(length - self.max_source_length, 0)
            x.input_ids = x.input_ids[:, start:]
            x.attention_mask = x.attention_mask[:, start:]
            assert x.input_ids.shape[1] == self.max_source_length

        return x

    def _getitem_(self, i: int) -> dict:
        x = self.tokenize(self.data.samples[i].source, is_source=True)
        y = self.tokenize(self.data.samples[i].target, is_source=False)

        return {
            "source_ids": x.input_ids.squeeze(),
            "source_mask": x.attention_mask.squeeze(),
            "target_ids": y.input_ids.squeeze(),
            "target_mask": y.attention_mask.squeeze(),
        }

    def to_human_readable(self, raw: dict) -> dict:
        source = self.tokenizer.decode(raw["source_ids"])
        target = self.tokenizer.decode(raw["target_ids"])
        return dict(source=source, target=target)


def preprocess_mental(
    path_in: str = "data/mental.json", path_out: str = "data/train.json"
):
    data = Mental_Data.load(path_in).as_data()
    data.analyze()
    data.save(path_out)


# Finetuning Model

In [4]:
import sys

from data_loading import TextToTextDataset

# os.environ["TOKENIZERS_PARALLELISM"] = "true"

def get_args(raw_args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name_or_path", type=str, default="google-t5/t5-base")
    parser.add_argument("--max_source_length", type=int, default=40)
    parser.add_argument("--max_target_length", type=int, default=160)
    parser.add_argument("--data_path", type=str)#, default="data/train.json")
    parser.add_argument("--train_epochs", type=int, default=5)
    parser.add_argument("--train_batch_size", type=int, default=16)
    parser.add_argument("--gradient_accumulation_steps", type=int, default=1)
    parser.add_argument("--learning_rate", type=float, default=3e-3)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--weight_decay", type=float, default=0.0)
    parser.add_argument("--output_dir", type=str, default="")
    parser.add_argument("--use_compile", action="store_true")
    parser.add_argument("--use_gradient_checkpointing", action="store_true")
    parser.add_argument("--use_fsdp", action="store_true")
    parser.add_argument("--use_lora", action="store_true")
    parser.add_argument("--debug", action="store_true")

    args = parser.parse_args(raw_args)
    return args



class LightningModel(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        print(self.hparams)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            self.hparams.model_name_or_path
        )
        print(dict(orig_state_dict=len(self.model.state_dict())))
        if self.hparams.use_lora:
            # https://github.com/huggingface/peft/blob/main/examples/conditional_generation/peft_lora_seq2seq.ipynb
            peft_config = LoraConfig(
                task_type=TaskType.SEQ_2_SEQ_LM,
                inference_mode=False,
                r=8,
                lora_alpha=16,
                lora_dropout=0.1,
            )
            self.model = get_peft_model(self.model, peft_config)
        if self.hparams.use_compile:
            self.model = torch.compile(self.model)
        if self.hparams.use_gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.tokenizer = AutoTokenizer.from_pretrained(self.hparams.model_name_or_path)

    def forward(
        self,
        input_ids,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        labels=None,
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=lm_labels,
            decoder_attention_mask=batch["target_mask"],
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("loss", loss, on_step=True, prog_bar=True, rank_zero_only=True)
        return loss

    def configure_optimizers(self):
        no_decay = ["bias", "LayerNorm.weight"]
        params = self.trainer.model.named_parameters()
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in params if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in params if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

        # noinspection PyTypeChecker
        optimizer = Adafactor(
            optimizer_grouped_parameters,
            lr=self.hparams.learning_rate,
            relative_step=False,
        )
        return [optimizer]

    def train_dataloader(self):
        dataset = TextToTextDataset(
            path=self.hparams.data_path,
            max_source_length=self.hparams.max_source_length,
            max_target_length=self.hparams.max_target_length,
            tokenizer=self.tokenizer,
        )

        return DataLoader(
            dataset,
            batch_size=self.hparams.train_batch_size,
            drop_last=True,
            shuffle=True,
        )

def main(args):
    torch.set_float32_matmul_precision("high")
#     args = get_args(raw_args)
    seed_everything(args.seed)
    model = LightningModel(args)

    saver = ModelCheckpoint(
        verbose=True,
        dirpath=args.output_dir,
        save_weights_only=True,
    )

    strategy = "auto"
    if args.use_fsdp:
        # https://pytorch.org/blog/efficient-large-scale-training-with-pytorch/
        # https://lightning.ai/docs/pytorch/stable/advanced/model_parallel.html
        strategy = MyFSDPStrategy(
            auto_wrap_policy=functools.partial(
                transformer_auto_wrap_policy,
                transformer_layer_cls={T5Block},
            ),
            mixed_precision=MixedPrecision(
                param_dtype=torch.float16,
                reduce_dtype=torch.float16,
                buffer_dtype=torch.float16,
            ),
            activation_checkpointing=T5Block,
            cpu_offload=True,
        )

    trainer = pl.Trainer(
        precision="32",
        accelerator="cuda",#cuda",
        strategy=strategy,
        accumulate_grad_batches=1 if args.debug else args.gradient_accumulation_steps,
        default_root_dir=args.output_dir,
        gradient_clip_val=None if args.use_fsdp else 1.0,
        max_epochs=args.train_epochs,
        callbacks=[saver],
        logger=False,
        overfit_batches=10 if args.debug else 0,
    )

    trainer.fit(model)
    return model

# Testing Model

In [5]:
import shutil
from pathlib import Path

import torch
from fire import Fire
from huggingface_hub import HfApi
from lightning_fabric import seed_everything

# from training import LightningModel


def test_model(path, prompt, max_length: int = 160, device: str = "cpu",):
    if not prompt:
        prompt = "Write a short email to show that 42 is the optimal seed for training neural networks"

    model: LightningModel = LightningModel.load_from_checkpoint(path)
    tokenizer = model.tokenizer
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    seed_everything(model.hparams.seed)
    with torch.inference_mode():
        model.model.eval()
        model = model.to(device)
        input_ids = input_ids.to(device)
        outputs = model.model.generate(
            input_ids=input_ids, max_length=max_length, do_sample=True
        )

    print(tokenizer.decode(outputs[0]))


def export_checkpoint(path: str, path_out: str):
    model = LightningModel.load_from_checkpoint(path)
    model.model.save_pretrained(path_out)
    model.tokenizer.save_pretrained(path_out)


def export_to_hub(path: str, repo: str, temp: str = "temp"):
    if Path(temp).exists():
        shutil.rmtree(temp)

    model = LightningModel.load_from_checkpoint(path)
    model.model.save_pretrained(temp)
    model.tokenizer.save_pretrained(temp)
    del model  # Save memory?

    api = HfApi()
    api.create_repo(repo_id=repo, repo_type="model", exist_ok=True)
    api.upload_folder(repo_id=repo, folder_path=temp)



# Calling Preprocessing function

In [6]:

preprocess_mental (path_in ="mental_health_counseling_conversations.json",path_out="kaggle/working/mental_processed.json")

{
  "Context_Sample": 3512,
  "Response_Sample": 3512
}
{
  "source": "I have bipolar II disorder, I'm addicted to alcohol and weed, and I'm hopeless. I keep drinking even though it's harming myself and others.",
  "target": "You may feel hopeless but YOU are not hopeless. Addictions of any sort are difficult to overcome, especially when they serve to bury pain and suffering that one is experiencing. It is not impossible to overcome alcohol or drug use/abuse/dependence on your own, but you will likely find much greater success with the help of a therapist or other support system such as rehabilitation or Alcoholics Anonymous. I suggest doing a bit of research to see what type of help\u00a0is available and feasible for you in your area and go from there. Know that recovery takes time, willingness, and effort. Don't give up and remember that you are not hopeless. You can make the choice to change your habits and learn new ways of healthy coping. Best of luck to you!"
}
{
  "source": "I'm

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

{
  "total_samples": 3512,
  "source": "{'min': 6, 'max': 695, 'median': 60, '95_percentile': 176}",
  "target": "{'min': 1, 'max': 27779, 'median': 186, '95_percentile': 571}"
}


# Calling Finetuning functions

In [14]:
args = get_args(["--model_name_or_path", "google-t5/t5-base","--train_epochs","10", "--data_path","kaggle/working/mental_processed.json","--learning_rate","0.005", "--output_dir", "kaggle/working/"])

model = main(args)


INFO:lightning_fabric.utilities.seed:Global seed set to 42


"data_path":                   kaggle/working/mental_processed.json
"debug":                       False
"gradient_accumulation_steps": 1
"learning_rate":               0.005
"max_source_length":           40
"max_target_length":           160
"model_name_or_path":          google-t5/t5-base
"output_dir":                  kaggle/working/
"seed":                        42
"train_batch_size":            16
"train_epochs":                10
"use_compile":                 False
"use_fsdp":                    False
"use_gradient_checkpointing":  False
"use_lora":                    False
"weight_decay":                0.0
{'orig_state_dict': 260}


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/kaggle/working exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# Evaluating BLEU Score

In [15]:
from datasets import load_dataset

import torch
from nltk.translate.bleu_score import corpus_bleu


# Load the evaluation dataset
eval_dataset = TextToTextDataset(
    path=args.data_path,
    tokenizer=model.tokenizer,
    max_source_length=args.max_source_length,
    max_target_length=args.max_target_length,
)

# Define the number of samples to remove
num_samples_to_remove = 3500  # Assuming this is the number of samples to remove
eval_dataset.data.samples = eval_dataset.data.samples[500:520]


# Define a function to generate responses using the model
def generate_responses(model, dataset):
    generated_responses = []
    for i in range(len(dataset)):
        input_ids = dataset[i]["source_ids"].unsqueeze(0)
        attention_mask = dataset[i]["source_mask"].unsqueeze(0)
        output_ids = model.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=args.max_target_length,
            num_beams=4,
            early_stopping=True,
            decoder_start_token_id=model.tokenizer.pad_token_id,
        )
        generated_responses.append(output_ids.squeeze().tolist())
    return generated_responses

# Generate responses using the finetuned model
generated_responses = generate_responses(model, eval_dataset)

# Convert generated responses back to text
generated_texts = [model.tokenizer.decode(ids, skip_special_tokens=True) for ids in generated_responses]

# Load reference target texts
target_texts = [sample.target for sample in eval_dataset.data.samples]

# Compute BLEU score
bleu_score = corpus_bleu([[text] for text in target_texts], generated_texts)

print("BLEU Score:", bleu_score)

kaggle/working/mental_processed.json: 100%|██████████| 3512/3512 [00:00<00:00, 67255.02it/s]


BLEU Score: 0.31400188365605675


# Testing Finetuned Model Output

In [16]:
test_model(path="kaggle/working/epoch=9-step=2190.ckpt",prompt="I have so many issues to address. I have a history of sexual abuse")

"data_path":                   kaggle/working/mental_processed.json
"debug":                       False
"gradient_accumulation_steps": 1
"learning_rate":               0.003
"max_source_length":           40
"max_target_length":           160
"model_name_or_path":          google-t5/t5-base
"output_dir":                  kaggle/working/
"seed":                        42
"train_batch_size":            16
"train_epochs":                10
"use_compile":                 False
"use_fsdp":                    False
"use_gradient_checkpointing":  False
"use_lora":                    False
"weight_decay":                0.0
{'orig_state_dict': 260}


INFO:lightning_fabric.utilities.seed:Global seed set to 42


<pad> Just go! You choose the topics that are easiest. Reading through your symptoms can help you decide which issues to tackle first. Also, if you have multiple issues, read through your symptoms as well. Once you feel comfortable with your counselor and choose a counselor, read about the symptoms and decide how to start working toward your goals. Sometimes, working on one issue helps to bring other issues to light in the context of your main problem.</s>


In [17]:
test_model(path="kaggle/working/epoch=9-step=2190.ckpt",prompt="I constantly feel like I'm not good enough, no matter what I do. I doubt my abilities and question my worth as a person. It's affecting my relationships and my work, and I just can't shake this feeling of being worthless.")

"data_path":                   kaggle/working/mental_processed.json
"debug":                       False
"gradient_accumulation_steps": 1
"learning_rate":               0.003
"max_source_length":           40
"max_target_length":           160
"model_name_or_path":          google-t5/t5-base
"output_dir":                  kaggle/working/
"seed":                        42
"train_batch_size":            16
"train_epochs":                10
"use_compile":                 False
"use_fsdp":                    False
"use_gradient_checkpointing":  False
"use_lora":                    False
"weight_decay":                0.0
{'orig_state_dict': 260}


INFO:lightning_fabric.utilities.seed:Global seed set to 42


<pad> It sounds like you are feeling pretty overwhelmed with self-doubt and self-worth, which is affecting your relationships and your work. The first step is to acknowledge that you are not "good enough", and instead see yourself as someone who can and does provide you with the qualities and qualities you want.It is possible that based on your positive self-doubt, the feelings of not being worthless, will come back as a way of coping with the underlying fear of not being worth enough, or at least, will lead to a decrease in self-doubt and self-worth. The second step is to start recognizing what you "worth" and then addressing the underlying fear with your therapist; 


In [18]:
test_model(path="kaggle/working/epoch=9-step=2190.ckpt",prompt="I feel like everything bad that happens is my fault. No matter what happens, I can't shake this overwhelming sense of guilt. It's like I'm carrying around this heavy burden of blame all the time, and I don't know how to let it go.")

"data_path":                   kaggle/working/mental_processed.json
"debug":                       False
"gradient_accumulation_steps": 1
"learning_rate":               0.003
"max_source_length":           40
"max_target_length":           160
"model_name_or_path":          google-t5/t5-base
"output_dir":                  kaggle/working/
"seed":                        42
"train_batch_size":            16
"train_epochs":                10
"use_compile":                 False
"use_fsdp":                    False
"use_gradient_checkpointing":  False
"use_lora":                    False
"weight_decay":                0.0
{'orig_state_dict': 260}


INFO:lightning_fabric.utilities.seed:Global seed set to 42


<pad> Sorry to hear you are feeling this way. I would encourage you to slowly process the feelings and thoughts that you have of the blame and what you feel is contributing to the guilt. Some people blame themselves for totally "bad things". There are many reasons for feeling this way. A major reason for feeling this way is that it is the feeling of blame you are carrying around with you everywhere you go. It is not unusual for feeling blamed when feeling something bad. When feeling something bad, it's usually the result of a bad event or a bad event. The good news is that usually feeling blame is releasing the person's guilt, and allowing the person's guilt to reassess what he/she feels.If you feel this way


In [19]:
test_model(path="kaggle/working/epoch=9-step=2190.ckpt",prompt="I used to love doing things that brought me joy, but lately, I just don't feel interested in anything. Nothing seems to bring me pleasure anymore, and I find myself going through the motions without really enjoying anything. It's like I've lost all sense of purpose or motivation.")

"data_path":                   kaggle/working/mental_processed.json
"debug":                       False
"gradient_accumulation_steps": 1
"learning_rate":               0.003
"max_source_length":           40
"max_target_length":           160
"model_name_or_path":          google-t5/t5-base
"output_dir":                  kaggle/working/
"seed":                        42
"train_batch_size":            16
"train_epochs":                10
"use_compile":                 False
"use_fsdp":                    False
"use_gradient_checkpointing":  False
"use_lora":                    False
"weight_decay":                0.0
{'orig_state_dict': 260}


INFO:lightning_fabric.utilities.seed:Global seed set to 42


<pad> Hello. Thank you for your question. Clearly this is something that has brought you happiness and also some great insight into what may be going on. I think you have identified some basic issues here. For instance, did you eat well, did you exercise, or did you get up and did you get up and did you eat well? These things should assist you when you are feeling down. In the mean time, do you exercise, or do you just take a walk around the block with no purpose? Maybe a hobby can bring you pleasure and you can connect with someone who can.There are many ways to enjoy your time, and the list of things you can do to increase your enjoyment. For example, if you value a certain activity or activity, you
